In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/air-quality-and-health-impact-dataset/air_quality_health_impact_data.csv')
pd.set_option('max_colwidth', 100)
df.head(5)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Özellikler ve hedef değişkeni belirleme
X = df.drop(columns=['HealthImpactClass'])
y = df['HealthImpactClass']

# Veri setini eğitim ve test olarak ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=58)

## 1) Boosting

Boosting, zayıf öğrenicileri (weak learners) güçlü bir tahminciye dönüştürmeyi amaçlayan bir makine öğrenimi topluluk (ensemble) tekniğidir. Boosting, her bir zayıf öğrenicinin hatalarını düzelterek modelin doğruluğunu artırır.

Boosting'in prensipleri şunlardır:

- Adım Adım Öğrenme: Boosting, zayıf öğrenicileri ardışık olarak eğitir. Her yeni model, önceki modellerin hatalarını düzelterek daha iyi tahmin yapmaya çalışır.
- Ağırlıklandırma: Her bir örneğin hata oranına göre ağırlıklandırılır. Hatalı sınıflandırılmış örnekler daha yüksek ağırlık alır, böylece sonraki model bu örnekleri daha iyi öğrenir.
- Topluluk Kararı: Nihai tahmin, tüm zayıf öğrenicilerin tahminlerinin ağırlıklı toplamı veya çoğunluk kararı ile belirlenir.

### 1.1 Gradient Boosting

Gradient Boosting, zayıf modellerin ardışık olarak eğitilmesiyle güçlü bir model oluşturur. Her bir model, önceki modellerin hatalarını düzelterek öğrenir.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Gradient Boosting modelini oluşturma
gb_model = GradientBoostingClassifier(random_state=58)
gb_model.fit(X_train, y_train)

# Tahmin yapma
y_pred = gb_model.predict(X_test)

# Sonuçları değerlendirme
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

### 1.2 AdaBoost

AdaBoost, zayıf öğreniciler (genellikle decision tree'ler) kullanarak ardışık olarak modeller oluşturur ve her adımda hataları düzelterek güçlü bir model oluşturur.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

# AdaBoost modelini oluşturma
ab_model = AdaBoostClassifier(random_state=58)
ab_model.fit(X_train, y_train)

# Tahmin yapma
y_pred = ab_model.predict(X_test)

# Sonuçları değerlendirme
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

### 1.3 XGBoost

XGBoost, performans ve hız açısından oldukça güçlü bir boosting algoritmasıdır. Aynı zamanda eksik verilerle başa çıkma yeteneği de vardır.

In [ ]:
import xgboost as xgb

# XGBoost modelini oluşturma
xgb_model = xgb.XGBClassifier(random_state=58)
xgb_model.fit(X_train, y_train)

# Tahmin yapma
y_pred = xgb_model.predict(X_test)

# Sonuçları değerlendirme
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

## 2) Majority Voting

Majority voting ensemble, birden fazla makine öğrenimi modelinin tahminlerini birleştirerek nihai sınıflandırma kararını veren bir ensemble yöntemidir. Bu yöntemde, her bir modelin verdiği sınıf tahminleri alınır ve en çok oy alan sınıf, nihai tahmin olarak seçilir. Bu yaklaşım, farklı modellerin güçlü yönlerini birleştirerek daha doğru ve sağlam bir tahmin yapmayı amaçlar.

Çalışma Prensibi
Farklı Modellerin Eğitimi: Farklı türde makine öğrenimi modelleri (örneğin, decision tree, k-nearest neighbors, logistic regression) aynı veri seti üzerinde eğitilir.
Tahminlerin Alınması: Test verileri üzerinde her bir modelin tahminleri yapılır.
Oyların Birleştirilmesi: Her bir veri noktası için modellerin yaptığı tahminler alınır ve en çok oyu alan sınıf, nihai tahmin olarak belirlenir.

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Veri setini yükleyin ve bağımlı ve bağımsız değişkenleri ayır
X = df.drop(columns=['HealthImpactClass'])
y = df['HealthImpactClass']

# Veri setini eğitim ve test olarak ayır
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=58)

# Modelleri tanımla
log_clf = LogisticRegression(random_state=58, max_iter=10000)
knn_clf = KNeighborsClassifier(n_neighbors=5)
dt_clf = DecisionTreeClassifier(random_state=58)

# Majority Voting Ensemble oluştur
voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('knn', knn_clf), ('dt', dt_clf)],
    voting='hard'  # 'hard' majority voting, 'soft' probability averaging
)

# Ansambl modelini eğit
voting_clf.fit(X_train, y_train)

# Tahmin yapın
y_pred = voting_clf.predict(X_test)

# Sonuçları değerlendir
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

## 3) Bagging (Bootstrap Aggregating)

Bagging yöntemi, modelin genel performansını artırmak amacıyla birden fazla modelin aynı veri setinden rastgele seçilmiş alt örnekler üzerinde eğitildiği bir ensemble yöntemidir. Bagging, overfitting riskini azaltarak modelin genelleme yeteneğini artırır. Bu yöntem, özellikle yüksek varyansa sahip modellerde etkili sonuçlar verir.
Bu yönteme örnek olarak Random Forest’ı ayrı bir .ipynb dosyasında kullandım. Random Forest da bir bagging yöntemidir.

## 4) Stacking

Stacking, birden fazla modelin tahminlerini bir araya getirerek, nihai tahmini yapmak için bu tahminleri ikinci bir modelle (meta-learner) öğrenen bir ensemble yöntemidir. Stacking, temel öğrenicilerin (base learners) zayıf yanlarını telafi ederek genel model performansını artırmayı amaçlar.

Stacking'in Adımları
Temel Öğrenicilerin Eğitimi (Base Learners):

Farklı algoritmalardan bir dizi temel model (örneğin, lojistik regresyon, SVM, karar ağaçları) eğitilir.
Bu modeller veri setinin farklı özelliklerini yakalamaya çalışır.
Meta Öğrenicinin Eğitimi (Meta-Learner):

Temel öğrenicilerin tahminleri bir araya getirilir ve bu tahminler, yeni bir model (meta-learner) ile birleştirilir.
Meta-learner, temel öğrenicilerin tahminlerine dayanarak nihai tahmini yapar.

Stacking, çok sınıflı veri setlerinde farklı model türlerinin güçlü yönlerini bir araya getirerek daha iyi bir genel performans sağlayabilir. Özellikle bu veri setinde, çeşitli hava kalitesi ve sağlık etkisi ölçümlerine dayanan çok sayıda sınıf bulunduğundan, stacking gibi bir ensemble yöntemi, sınıflandırma performansını artırmada etkili olabilir.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Veri setini yükleme
# df = pd.read_csv('path_to_dataset.csv')

# Özellikler ve hedef değişkeni belirleme
X = df.drop(columns=['HealthImpactClass'])
y = df['HealthImpactClass']

# Veri setini eğitim ve test olarak ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=58)

# Temel öğrenicileri tanımlama
estimators = [
    ('lr', LogisticRegression(max_iter=1000, random_state=58)),
    ('knn', KNeighborsClassifier(n_neighbors=5)),
    ('dt', DecisionTreeClassifier(random_state=58))
]

# StackingClassifier'ı tanımlama
stacking_clf = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(max_iter=1000, random_state=58),
    cv=5
)

# Modeli eğitme
stacking_clf.fit(X_train, y_train)

# Tahmin yapma
y_pred = stacking_clf.predict(X_test)

# Sonuçları değerlendirme
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))